In [12]:
import json
schema = {"input":[{"type":"string","name":"encounters_encounterclass"},{"type":"string","name":"patient_gender"},{"type":"string","name":"patient_marital"},{"type":"string","name":"patient_ethnicity"},{"type":"string","name":"patient_race"},{"type":"string","name":"providers_speciality"},{"type":"string","name":"encounters_reasoncode"},{"type":"string","name":"encounters_code"},{"type":"string","name":"procedures_code"},{"type":"double","name":"patient_healthcare_expenses"},{"type":"double","name":"patient_healthcare_coverage"},{"type":"double","name":"encounters_total_claim_cost"},{"type":"double","name":"encounters_payer_coverage"},{"type":"double","name":"encounters_base_encounter_cost"},{"type":"double","name":"procedures_base_cost"},{"type":"long","name":"providers_utilization"},{"type":"double","name":"age"}],"output":{"type":"double","name":"features","struct":"vector"}}
schema_json = json.dumps(schema)
print(schema_json)


{"input": [{"type": "string", "name": "encounters_encounterclass"}, {"type": "string", "name": "patient_gender"}, {"type": "string", "name": "patient_marital"}, {"type": "string", "name": "patient_ethnicity"}, {"type": "string", "name": "patient_race"}, {"type": "string", "name": "providers_speciality"}, {"type": "string", "name": "encounters_reasoncode"}, {"type": "string", "name": "encounters_code"}, {"type": "string", "name": "procedures_code"}, {"type": "double", "name": "patient_healthcare_expenses"}, {"type": "double", "name": "patient_healthcare_coverage"}, {"type": "double", "name": "encounters_total_claim_cost"}, {"type": "double", "name": "encounters_payer_coverage"}, {"type": "double", "name": "encounters_base_encounter_cost"}, {"type": "double", "name": "procedures_base_cost"}, {"type": "long", "name": "providers_utilization"}, {"type": "double", "name": "age"}], "output": {"type": "double", "name": "features", "struct": "vector"}}


In [13]:
from time import gmtime, strftime
import time

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

import sagemaker
from sagemaker import get_execution_role
from sagemaker.sparkml.model import SparkMLModel

sess = sagemaker.Session()
role = get_execution_role()

s3_model_bucket = "readmission-data-ehr"
s3_model_bucket_prefix = "spark-ml-model/2020/3/7"

# S3 location of where you uploaded your trained and serialized SparkML model
sparkml_data = 's3://{}/{}/{}'.format(s3_model_bucket, s3_model_bucket_prefix, 'model.tar.gz')
model_name = 'sparkml-abalone-' + timestamp_prefix
sparkml_model = SparkMLModel(model_data=sparkml_data, 
                             role=role, 
                             sagemaker_session=sess, 
                             name=model_name,
                             # passing the schema defined above by using an environment 
                             #variable that sagemaker-sparkml-serving understands
                             env={'SAGEMAKER_SPARKML_SCHEMA' : schema_json})


endpoint_name = 'sparkml-abalone-ep-' + timestamp_prefix
sparkml_model.deploy(initial_instance_count=1, instance_type='ml.c4.xlarge', endpoint_name=endpoint_name)

-----------------!

In [14]:
from sagemaker.predictor import json_serializer, csv_serializer, json_deserializer, RealTimePredictor
from sagemaker.content_types import CONTENT_TYPE_CSV, CONTENT_TYPE_JSON
payload = "outpatient,M,S,hispanic,white,F,271737000,185347001,430193006,262241.40,2324.88,129.16,64.16,129.16,526.17,16,40"
predictor = RealTimePredictor(endpoint=endpoint_name, sagemaker_session=sess, serializer=csv_serializer,
                                content_type=CONTENT_TYPE_CSV, accept=CONTENT_TYPE_CSV)
print(predictor.predict(payload))

b'0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [15]:
boto_session = sess.boto_session
sm_client = boto_session.client('sagemaker')
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '78de6b9f-1938-4429-a5f3-793045509666',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '78de6b9f-1938-4429-a5f3-793045509666',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 07 Mar 2020 07:24:23 GMT'},
  'RetryAttempts': 0}}